In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
import os 


model = os.environ["LLM_MODEL_NAME"]
api_key = os.environ["LLM_API_KEY"]
base_url = os.environ["OLLAMA_ENDPOINT"] 

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

# Get the fetch tool from mcp-server-fetch.
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"])
tools = await mcp_server_tools(fetch_mcp_server)

# Create an agent that can use the fetch tool.
model_client = OpenAIChatCompletionClient(
    model=model,
    base_url=base_url,
    api_key=api_key,
    model_info={
        "family": "llama",
        "vision": False,
        "json_output": True,
        "function_calling": True,
    },
)
agent = AssistantAgent(name="fetcher", model_client=model_client, tools=tools, reflect_on_tool_use=True)  # type: ignore

# Let the agent fetch the content of a URL and summarize it.
result = await agent.run(
    task="Summarize the content of https://en.wikipedia.org/wiki/Seattle"
)
print(result.messages[-1].content)

The provided URL does not seem to have been processed correctly.

To summarize the content of https://en.wikipedia.org/wiki/Seattle:

The Seattle article on Wikipedia covers a wide range of topics related to the city, including its history, geography, climate, education, economy, culture, and attractions. Here are some key points:

- History: Seattle was founded in 1851 and grew rapidly due to its favorable location near the Puget Sound and the Cascade Mountains.
- Geography: The city lies on a narrow isthmus between Puget Sound and Lake Washington.
- Climate: Seattle's climate is mild and temperate, with rainfall throughout the year.


In [ ]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))
 
agent = AssistantAgent(
    "assistant", tools=[tool], model_client=model_client, system_message="Use the `df` variable to access the dataset."
)
await Console(
    agent.on_messages_stream(
        [TextMessage(content="What's the average age of the passengers?", source="user")], CancellationToken()
    ),
    output_stats=True,
)

In [ ]:
from typing import Literal

from pydantic import BaseModel


# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]
    


model_client = OpenAIChatCompletionClient(
    model=model,
    api_key=api_key,
    base_url=base_url,
    model_info={
        "family": "llama",
        "vision": False,
        "json_output": True,
        "function_calling": True,
    },
    response_format=AgentResponse
)
    

agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="Categorize the input as happy, sad, or neutral following the JSON format.",
    model_client_stream= True
)

await Console(agent.run_stream(task="I am happy."))


---------- user ----------
I am happy.
---------- assistant ----------
{
  "thoughts": "You are happy",
  "response": "happy"
}


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='I am happy.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=22), metadata={}, content='{\n  "thoughts": "You are happy",\n  "response": "happy"\n}', type='TextMessage')], stop_reason=None)

In [9]:
streaming_assistant = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="You are a helpful assistant.",
    model_client_stream=True,  # Enable streaming tokens.
)


async for message in streaming_assistant.on_messages_stream(  # type: ignore
    [TextMessage(content="Name two cities in South America", source="user")],
    cancellation_token=CancellationToken(),
):
    print(message)


source='assistant' models_usage=None metadata={} content='{' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' ' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content='"' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content='t' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content='ho' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content='ugh' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content='t' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content='s' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content='":' type='ModelClientStreamingChunkEvent'
source='assistant' models_usage=None metadata={} content=' "' type='ModelClientStreamingChunkEvent'
sourc